# SHAPASH

In [4]:
pip install --upgrade pip

  Attempting uninstall: pip
    Found existing installation: pip 21.1.1
    Uninstalling pip-21.1.1:
      Successfully uninstalled pip-21.1.1
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
pip install lightgbm

In [8]:

import pandas as pd
from category_encoders import OrdinalEncoder
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor
from shapash import SmartExplainer


In [11]:
data = pd.read_csv('/Users/manal/Desktop/Manale/EICNAM/Python_S4_Moreau/depart_employe/depart_employes.csv', sep = ';')
display(data.head())

,Satisfaction,derniere_evaluation,Nombre_de_projets,Nombre_heures_mensuelles_moyenne,Temps_passe_dans_entreprise,Accident_du travail,depart,promotion_5_dernieres_annees,Service,niveau_salaire
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [12]:
y_df=data['depart'].to_frame()
X_df=data[data.columns.difference(['depart'])]

In [13]:
data.shape

data.head()

,Satisfaction,derniere_evaluation,Nombre_de_projets,Nombre_heures_mensuelles_moyenne,Temps_passe_dans_entreprise,Accident_du travail,depart,promotion_5_dernieres_annees,Service,niveau_salaire
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [14]:
#from category_encoders import OrdinalEncoder

categorical_features = [col for col in X_df.columns if X_df[col].dtype == 'object']

encoder = OrdinalEncoder(
    cols=categorical_features,
    handle_unknown='ignore',
    return_df=True).fit(X_df)

X_df=encoder.transform(X_df)

In [15]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X_df, y_df, train_size=0.75, random_state=1)

In [16]:
regressor = LGBMRegressor(n_estimators=100).fit(Xtrain,ytrain)

In [17]:
xpl = SmartExplainer(
    model=regressor,
)

In [18]:
xpl.compile(x=Xtest,
            y_target=ytest # Optional: allows to display True Values vs Predicted Values
           )

In [19]:
#affiche le dashboard sur navigateur

app = xpl.run_app(title_story='Départ Employés', port=8020)

c:\users\manal\appdata\local\programs\python\python39\lib\site-packages\shapash\webapp\smart_app.py:354: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.

INFO:root:Your Shapash application run on http://LAPTOP-L4G8DGH9:8020/


Dash is running on http://0.0.0.0:8020/



INFO:root:Use the method .kill() to down your app.
INFO:dash.dash:Dash is running on http://0.0.0.0:8020/



 * Serving Flask app 'shapash.webapp.smart_app'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8020
 * Running on http://192.168.1.28:8020
INFO:werkzeug:Press CTRL+C to quit


In [20]:
##cloture dashboard

app.kill()

In [21]:
summary_df= xpl.to_pandas(
    max_contrib=3, # Number Max of features to show in summary
    #threshold=5000,
)

summary_df.head()

,pred,feature_1,value_1,contribution_1,feature_2,value_2,contribution_2,feature_3,value_3,contribution_3
11908,-0.035812,Satisfaction,0.67,-0.080764,derniere_evaluation,0.39,-0.078376,Nombre_heures_mensuelles_moyenne,207.0,-0.058829
10508,-0.008298,Satisfaction,0.51,-0.149531,Temps_passe_dans_entreprise,3.0,-0.048776,derniere_evaluation,0.98,-0.02255
9193,0.036048,Satisfaction,0.66,-0.200223,Nombre_de_projets,2.0,0.036972,Temps_passe_dans_entreprise,2.0,-0.031934
13087,0.001591,Satisfaction,0.53,-0.120554,Temps_passe_dans_entreprise,2.0,-0.051585,Nombre_de_projets,3.0,-0.032463
506,0.981898,Temps_passe_dans_entreprise,5.0,0.516511,derniere_evaluation,0.98,0.123793,Nombre_heures_mensuelles_moyenne,259.0,0.057198


In [ ]:
#compression format binaire doc pickle
#xpl.save('./xpl.pkl')